<a href="https://colab.research.google.com/github/jpcompartir/dl_notebooks/blob/main/core_keras_apis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras 
import tensorflow as tf

Keras is built around the layer abstraction. A layer is an object that contains some weights (a DL model's knowledge) and a forward pass.

Weights are typically defined in the build() method, but can also be defined in __init__(), the computation is defined in the call() method rather than __call__().

Thinking back to our NaiveDense class from previous chapter, which comprised two Weights W and bias b, and the output was given by activation(dot( input, W) + b) - an affine transformation - implemented as follows:

In [2]:
class SimpleDense(keras.layers.Layer):
    def __init__(self, units, activation = None ):
        super(SimpleDense, self).__init__() 
        self.units = units
        self.activation = activation
    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.W = self.add_weight(shape = (input_dim, self.units),
                                 initializer = "random_normal")
        self.b = self.add_weight(shape = (self.units,),
                                 initializer = "zeros")
    def call(self, inputs):
        y = tf.matmul(inputs, self.W)+ self.b
        if self.activation is not None:
          y = self.activation(y)
        return y

Why did we define call instead of making use of __call__()? To create the state just in time.

The SimpleDense class will now accept a tensorflow tensor (and other objcts) and an activation function. We can instantiate an object of the class as we'd call a function.

In [3]:
my_dense = SimpleDense(units=32, activation=tf.nn.relu)
input_tensor = tf.ones(shape=(2, 784))
output_tensor = my_dense(input_tensor)
print(output_tensor.shape)


(2, 32)


In [ ]:
input_tensor

<tf.Tensor: shape=(2, 784), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>

Let's see how Keras takes care of this through its layers & models abstraction.

In [5]:
from tensorflow.keras import layers

In [6]:
layer = layers.Dense(32, activation = "relu")

In [7]:
from tensorflow.keras import models

Layers will infer the shape of their inputs, rather than have them stated as we did above.

In [9]:
model = models.Sequential([
                           layers.Dense(32, activation = "relu"),
                           layers.Dense(32)
])

Recall the NaiveDense we implemented in an earlier notebook, in which we had to specify the input for each layer e.g.

In [ ]:
# model = NaiveSequntial([
#                         NaiveDense(input_size = 784, output_size =32, activation = "relu"),
#                         NaiveDense(input_size = 32, ... etc.)
# ])

In [10]:
model = keras.Sequential([
                          SimpleDense(32, activation = "relu"),
                          SimpleDense(64, activation = "relu"),
                          SimpleDense(32, activation = "relu"),
                          SimpleDense(10, activation = "softmax")
])

Recall in the Keras API we define our layers, then we compile our model with a loss function (representation of distance between prediction & label), optimiser (for automatic differentiation & back propagation) & metric (for validating our mdoel's performance):

In [13]:
model = keras.Sequential([keras.layers.Dense(1)])


In [15]:
model.compile(optimizer = "rmsprop",
              loss = "mean_squared_erorr",
              metrics = ["accuracy"])

Identical is instantiating with objects:

In [16]:
model.compile(optimizer = keras.optimizers.RMSprop(),
              loss = keras.losses.MeanSquaredError(),
              metrics = [keras.metrics.BinaryAccuracy()])

Why do that? In the case of passing in custom loss functions & metrics (when advanced user), or if we want to customise the objects by setting arguments e.g.

In [17]:
# model.compile(optimizer = keras.optimizers.RMSpop(learning_rate = 1e-4),
#               loss = my_custom_loss,
#               metrics = [my_custom_metric_1, my_custom_metric_2]

Examples of optimizers: SGD, RMSpop, Adam, Adagrad etc. (RMSpop usually good enough, Adam very popular) etc. 
Losses: CategoricalCrossEntropy (classification), SparseCategoricalCrossEntropy, BinaryCrossEntropy, MeanSquaredError, KLDivergence, CosineSimilarity
Metrics: CategoricalAccuracy, SparseCategoricalAccuracy, BinaryAccuracy, AUC, Precision, Recall etc.

---

After compiling the model, it's time to define the training loop, using the fit() method. In this case we'll specify the inputs, targets, epochs, batch_size

In [18]:
#Won't run as we haven't defined inputs, targets etc.
history = model.fit(
    inputs,
    targets,
    epochs =5,
    batch_size = 128
)

NameError: ignored